In [23]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, NMF
import pickle
from sklearn.feature_extraction import text
import numpy as np


In [3]:
df = pd.read_csv('/Users/augustinebowe/Downloads/archive-3/winemag-data.csv')
df.shape

(150930, 11)

In [4]:
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df.description = df.description.map(alphanumeric).map(punc_lower)
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,this tremendous varietal wine hails from oa...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,ripe aromas of fig blackberry and cassis are ...,Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,mac watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,this spent months in new french oak and ...,Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,this is the top wine from la bégude named aft...,La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [18]:
df.description = df.description.map(alphanumeric).map(punc_lower)
df['wine-name'] = df['winery'] + ' ' + df['variety'] + ' ' + df['designation']
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,wine-name
0,0,US,this tremendous varietal wine hails from oa...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Heitz Cabernet Sauvignon Martha's Vineyard
1,1,Spain,ripe aromas of fig blackberry and cassis are ...,Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,Bodega Carmen Rodríguez Tinta de Toro Carodoru...
2,2,US,mac watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Macauley Sauvignon Blanc Special Selected Late...
3,3,US,this spent months in new french oak and ...,Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Ponzi Pinot Noir Reserve
4,4,France,this is the top wine from la bégude named aft...,La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,Domaine de la Bégude Provence red blend La Brû...


In [5]:
X = df.description
print(df.shape)
print(sum([len(d.split(' ')) for d in X]) > 100000)

(150930, 11)
True


In [6]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [9]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(["wine","wines","flavor","flavors","aroma","aromas","palate"])

In [10]:
cv = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2),
                     min_df = .001,
                     max_df = .95)
X_cv = cv.fit_transform(X)
dtm_cv = pd.DataFrame(X_cv.toarray(), columns=cv.get_feature_names())
dtm_cv.head()

,abound,absolutely,abundant,acacia,accent,accented,accenting,accents,acceptable,accessible,...,zealand,zest,zesty,zesty acidity,zin,zinfandel,zingy,zins,zippy,zweigelt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
pca = PCA(n_components = 300)
pca.fit(dtm_cv)

PCA(n_components=300)

In [28]:
pca_components = pca.components_
pca_components
total_variance = sum(pca.explained_variance_ratio_)
total_variance

0.5714461292910439

In [8]:
tv = TfidfVectorizer(stop_words='english', ngram_range=(1,2),
                    min_df = .001,
                    max_df = .5,)
X_tv = tv.fit_transform(X)
dtm_tv = pd.DataFrame(X_tv.toarray(), columns=tv.get_feature_names())
dtm_tv.head()

,abound,absolutely,abundant,acacia,accent,accented,accenting,accents,acceptable,accessible,...,zealand,zest,zesty,zesty acidity,zin,zinfandel,zingy,zins,zippy,zweigelt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
pca_tv = PCA(n_components = 300)
pca_tv.fit(dtm_tv)


PCA(n_components=300)

In [30]:
total_variance_tv = sum(pca_tv.explained_variance_ratio_)
total_variance_tv

0.3509772256366196

In [11]:
nmf_cv = NMF(n_components = 300)
nmf_cv.fit(dtm_cv)

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


NMF(n_components=300)

In [16]:
reduced_matrix = nmf_cv.transform(dtm_cv)
reduced_matrix

array([[0.00381417, 0.        , 0.02960196, ..., 0.        , 0.08245629,
        0.00052711],
       [0.        , 0.        , 0.        , ..., 0.00071554, 0.0005931 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00499206],
       [0.        , 0.        , 0.04915113, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07762932, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [57]:
rows = df['wine-name']
red_df = pd.DataFrame(reduced_matrix,index=rows)
red_df.index
red_df1 = red_df[red_df.index.notnull()]
red_df1.shape

(105195, 300)

In [12]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [13]:
display_topics(nmf_cv, cv.get_feature_names(), 5)


Topic  0
tannins, soft tannins, dry tannins, astringent, smooth tannins

Topic  1
acidity, crisp acidity, fresh acidity, bright acidity, brisk

Topic  2
fruit, dark fruit, passion fruit, passion, blackberry fruit

Topic  3
finish, finish drink, short, long finish, turns

Topic  4
red, red cherry, red fruits, red plum, red currant

Topic  5
cherry, blackberry cherry, red cherry, cherry blackberry, cherry berry

Topic  6
cabernet, cabernet sauvignon, merlot cabernet, blend cabernet, napa

Topic  7
ripe, ripe black, ripe blackberry, ripe cherry, ripe berry

Topic  8
black fruit, black, fruit, fruit spice, ripe black

Topic  9
oak, new oak, smoky oak, sweet oak, fruit oak

Topic  10
spice, fruit spice, vanilla spice, sweet spice, baking spice

Topic  11
berries, red berries, wild berries, plums, berries cherries

Topic  12
sweet, sweet fruit, slightly sweet, sugary, sweet oak

Topic  13
drink, finish drink, easy drink, drink years, tannins drink

Topic  14
velvety, velvety tannins, soft v

In [82]:
test_text = ["full-bodied with dark fruit, chocolate and coffee flavors"]
vector = cv.transform(test_text)
vector_red = nmf_cv.transform(vector)
vector_red

array([[0.00000000e+00, 0.00000000e+00, 4.90690375e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.30573487e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 4.55881216e-05, 2.33213399e-02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.89007902e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        6.01394438e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [83]:
def recommend(text):
    vector1 = cv.transform(text)
    vector2 = nmf_cv.transform(vector1)
    vector3 = vector2.T
    recs = []
    for item in range(red_df1.shape[0]):
            recs.append([item,np.dot(vector2,red_df1.iloc[item])])
    final_rec = [i[0] for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    ilocs = final_rec[:2]
    return([red_df1.iloc[i].name for i in ilocs])

print(recommend(test_text))

['Stark-Condé Cabernet Sauvignon Stark', 'Dr. Loosen Riesling Wehlener Sonnenuhr Auslese Gold Capsule']


In [39]:
np.dot(red_df.iloc[11],vector_red.T)

array([3.72266365e-07])

In [72]:
review = ["Inviting aromas of ripe plum violet baking spice and leather waft out of the glass. Full-bodied and dense the approachable palate features fleshy black cherry star anise and mocha while velvety tannins provide gentle support. It's already ready so drink soon."]

print(recommend(review))

['Masi Red Blend Costasera', 'Tenute Fiorebelli Red Blend Coste Bianche']


In [73]:
review1 = ["Opulent aromas of cooked apple lemon rind light praline and stone. Full-bodied with linear and lively acidity that runs through the wine. Energetic and bright with tightness and structure. Steely. Drinkable now but better in 2023."]
print(get_recommends(review1))

['Louis Latour Pinot Noir Les Chaillots Premier Cru', 'Pascal Doquet Chardonnay Le Mesnil sur Oger Grand Cru Blanc de Blancs Brut', 'Pascal Doquet Chardonnay Le Mesnil sur Oger Grand Cru Blanc de Blancs Brut']
